In [1]:
from mtasklite import Pool, delayed_init, is_exception, set_process_start_method, ExceptionBehaviour, ArgumentPassing
# Use of tqdm is optional!
from tqdm.auto import tqdm 

In [2]:
# It's really important to do so on Mac
set_process_start_method()

In [3]:
import multiprocessing as mp

@delayed_init
class SimpleArithmeticClassWorker:
    def __init__(self, proc_id):
        print(f'Initialized process {mp.current_process()} passed ID = {proc_id}\n')
        self.proc_id = proc_id

    def __call__(self, input_arg):
        ret_val = input_arg
        for t in range(100_000):
            ret_val = (ret_val * ret_val) % 337
        return ret_val

In [4]:
N_TASKS = 1000      
N_JOBS = 4
TIMEOUT=None # Set to some small number, e.g., 1e-5 to see the timeout exception happening
    
input_arr = [k * 10 for k in range(N_TASKS)]
function_or_worker_arr = [SimpleArithmeticClassWorker(proc_id) for proc_id in range(N_JOBS)]

In [5]:
def sample_func(input_arg):
    ret_val = input_arg
    for t in range(100_000):
        ret_val = (ret_val * ret_val) % 337
    return ret_val

In [10]:
tot_res = 0    

with Pool(function_or_worker_arr,
         argument_type=ArgumentPassing.AS_SINGLE_ARG,
         exception_behavior=ExceptionBehaviour.IGNORE,
         is_unordered=False,
         task_timeout=TIMEOUT,
         join_timeout=1) as proc_pool:

    for result in tqdm(proc_pool(input_arr)):
        # With deferred exceptions, they are simply returned as a part of the result object
        if is_exception(result):
            print('Error:', type(result), str(result))
        else:
            tot_res += result

print('Total:', tot_res, proc_pool.exited)

  0%|          | 0/1000 [00:00<?, ?it/s]

Initialized process <Process name='Process-17' parent=8302 started daemon> passed ID = 0
Initialized process <Process name='Process-18' parent=8302 started daemon> passed ID = 1
Initialized process <Process name='Process-19' parent=8302 started daemon> passed ID = 2
Initialized process <Process name='Process-20' parent=8302 started daemon> passed ID = 3




Total: 112152 True
